In [ ]:
from preprocessing import getdataset,character_to_number, number_to_character

MAX_LEN = 282

dataset , table = getdataset("character", MAX_LEN)
print(table.number_to_character(0))

<pad>


In [ ]:
from network import make_model

EMBED_DIM = 32
NUM_HEADS = 8
model = make_model(MAX_LEN, table.vocab_size, EMBED_DIM, NUM_HEADS)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 282)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding         │ (None, 282, 32)             │          16,544 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block (TransformerBlock) │ (None, 282, 32)             │          34,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_1                  │ (None, 282, 32)             │          34,752 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_2                  │ (None, 282, 32)             │          34,752 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_3                  │ (None, 282, 32)             │          34,752 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_4                  │ (None, 282, 32)             │          34,752 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_5                  │ (None, 282, 32)             │          34,752 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_6                  │ (None, 282, 32)             │          34,752 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_7                  │ (None, 282, 32)             │          34,752 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_8                  │ (None, 282, 32)             │          34,752 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_9                  │ (None, 282, 32)             │          34,752 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 282, 20)             │             660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 282, 235)            │           4,935 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 369,659 (1.41 MB)

 Trainable params: 369,659 (1.41 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["sparse_categorical_accuracy"])


In [ ]:
X = tf.convert_to_tensor(dataset)
Y = np.zeros(X.shape).astype(int)
Y[:,0:-1] = np.array(X[:,1:])
Y = tf.convert_to_tensor(Y)
print(X)
print(Y)
#Y = tf.one_hot(Y, depth=table.vocab_size)
# change X to one_hot_vectors

tf.Tensor(
[[ 2  3  4 ...  0  0  0]
 [ 2 14 10 ...  0  0  0]
 [ 2 14 10 ...  0  0  0]
 ...
 [ 2 20 12 ...  0  0  0]
 [ 2 20 12 ...  0  0  0]
 [ 2 20 12 ...  0  0  0]], shape=(43352, 282), dtype=int32)
tf.Tensor(
[[ 3  4  5 ...  0  0  0]
 [14 10 11 ...  0  0  0]
 [14 10 11 ...  0  0  0]
 ...
 [20 12 16 ...  0  0  0]
 [20 12 16 ...  0  0  0]
 [20 12 16 ...  0  0  0]], shape=(43352, 282), dtype=int64)


In [ ]:
history = model.fit(X, Y, epochs=20, batch_size=16)

Epoch 1/20
2710/2710 ━━━━━━━━━━━━━━━━━━━━ 220s 65ms/step - loss: 1.2995 - sparse_categorical_accuracy: 0.6861
Epoch 2/20
2710/2710 ━━━━━━━━━━━━━━━━━━━━ 162s 58ms/step - loss: 0.8024 - sparse_categorical_accuracy: 0.7707
Epoch 3/20
2710/2710 ━━━━━━━━━━━━━━━━━━━━ 201s 57ms/step - loss: 0.7352 - sparse_categorical_accuracy: 0.7891
Epoch 4/20
2710/2710 ━━━━━━━━━━━━━━━━━━━━ 202s 57ms/step - loss: 0.7071 - sparse_categorical_accuracy: 0.7965
Epoch 5/20
2710/2710 ━━━━━━━━━━━━━━━━━━━━ 201s 57ms/step - loss: 0.6877 - sparse_categorical_accuracy: 0.8016
Epoch 6/20
2710/2710 ━━━━━━━━━━━━━━━━━━━━ 156s 58ms/step - loss: 0.6747 - sparse_categorical_accuracy: 0.8048
Epoch 7/20
2710/2710 ━━━━━━━━━━━━━━━━━━━━ 156s 58ms/step - loss: 0.6716 - sparse_categorical_accuracy: 0.8059
Epoch 8/20
2710/2710 ━━━━━━━━━━━━━━━━━━━━ 201s 57ms/step - loss: 0.6559 - sparse_categorical_accuracy: 0.8100
Epoch 9/20
2710/2710 ━━━━━━━━━━━━━━━━━━━━ 202s 57ms/step - loss: 0.6519 - sparse_categorical_accuracy: 0.8110
Epoch 10/2

KeyboardInterrupt: 

In [ ]:

text_to_begin_with = "@"
begining = [ table.table[i] for i in text_to_begin_with ]
nb_to_repeat = MAX_LEN - len(begining) -1
input = np.array([[2] + begining + [0] * nb_to_repeat])
exit = False
nb_iter = len(text_to_begin_with)+1
max_iter = MAX_LEN

print( "".join( [table.number_to_character( input[0,i] ) for i in range(len(input[0]))] ) )

while not exit:

  to_print = []
  for i in range(MAX_LEN):
    to_print.append(table.number_to_character(input[0, i]))

  input_tf = tf.convert_to_tensor(input)

  decoded = model.predict(input_tf)
  print(decoded.shape, input_tf.shape)
  output = []

  for i in range(MAX_LEN):
    output.append((np.argmax(decoded[0][i])))

  print("added : ")
  print( output )
  input[0, (nb_iter+1)] = output[nb_iter]

  if 1 in input or nb_iter >= max_iter-1:
    exit = True
  print("".join(to_print))

  nb_iter += 1

print("".join(to_print))




<sos><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>